# **Install dan Setting Library**

- Setting selenium khusus Google Colab: https://nariyoo.com/python-how-to-run-selenium-in-google-colab/
- Tidak perlu run kode cell ini jika melakukan running di Jupyter Notebook atau local Python lain

In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (151 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
57 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping a

In [2]:
!pip3 install newspaper3k

In [3]:
!pip3 install chromedriver-autoinstaller

In [4]:
# Import library
import re
import csv
import sys
import time
import pandas as pd
import chromedriver_autoinstaller
from newspaper import Article
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [5]:
# Setting chrome driver
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()

'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/129/chromedriver'

# **Load CSV Link Berita**

In [6]:
from google.colab import files
upload = files.upload()

Saving link_cnbc.csv to link_cnbc (1).csv


In [7]:
# Import daftar link berita menjadi DataFrame
df_news_link = pd.read_csv('link_cnbc.csv')
df_news_link

,Title,Link,Date
0,"Kelas 1,2,3 Bakal Dihapus, Cek Iuran BPJS Kese...",https://www.cnbcindonesia.com/news/20240922222...,13 jam yang lalu
1,"Kelas BPJS 1, 2, 3 Dihapus Jokowi, Cek Iuran T...",https://www.cnbcindonesia.com/news/20240922080...,1 hari yang lalu
2,"BPJS Kesehatan Ungkap 3 RS Manipulasi Klaim, B...",https://www.cnbcindonesia.com/news/20240920071...,3 hari yang lalu
3,"Wajib Tahu, 5 Jenis Operasi Ini Tak Ditanggung...",https://www.cnbcindonesia.com/lifestyle/202409...,4 hari yang lalu
4,"RI Siap Punya BBM Baru Low Sulfur, Luhut Sebut...",https://www.cnbcindonesia.com/news/20240917140...,6 hari yang lalu
...,...,...,...
2464,BPJS Ketenagakerjaan Targetkan Dana Kelolaan R...,https://www.cnbcindonesia.com/market/201802231...,6 tahun yang lalu
2465,Apindo: Pengelolaan Pembiayaan Perumahan Bisa ...,https://www.cnbcindonesia.com/news/20180220123...,6 tahun yang lalu
2466,"Sudah Punya BPJS Kesehatan, Perlukah Miliki As...",https://www.cnbcindonesia.com/mymoney/20180207...,6 tahun yang lalu
2467,Mengintip Dana Kelolaan BPJS Ketenagakerjaan y...,https://www.cnbcindonesia.com/market/201802021...,6 tahun yang lalu


In [10]:
df_news_link = pd.read_csv('link_cnbc.csv')
df_news_link = df_news_link.head(1500)
df_news_link

,Title,Link,Date
0,"Kelas 1,2,3 Bakal Dihapus, Cek Iuran BPJS Kese...",https://www.cnbcindonesia.com/news/20240922222...,13 jam yang lalu
1,"Kelas BPJS 1, 2, 3 Dihapus Jokowi, Cek Iuran T...",https://www.cnbcindonesia.com/news/20240922080...,1 hari yang lalu
2,"BPJS Kesehatan Ungkap 3 RS Manipulasi Klaim, B...",https://www.cnbcindonesia.com/news/20240920071...,3 hari yang lalu
3,"Wajib Tahu, 5 Jenis Operasi Ini Tak Ditanggung...",https://www.cnbcindonesia.com/lifestyle/202409...,4 hari yang lalu
4,"RI Siap Punya BBM Baru Low Sulfur, Luhut Sebut...",https://www.cnbcindonesia.com/news/20240917140...,6 hari yang lalu
...,...,...,...
1495,Rekam Jejak Kenaikan Iuran BPJS Kesehatan Zama...,https://www.cnbcindonesia.com/news/20201221175...,3 tahun yang lalu
1496,"Transparansi, BPJS Kesehatan Luncurkan Data Sa...",https://www.cnbcindonesia.com/news/20201221193...,3 tahun yang lalu
1497,"Catat, Perjalanan Kenaikan Iuran BPJS Kesehata...",https://www.cnbcindonesia.com/news/20201221120...,3 tahun yang lalu
1498,"Catat! Jokowi Janji Vaksin Gratis Warga RI, Ta...",https://www.cnbcindonesia.com/news/20201219114...,3 tahun yang lalu


# **Scraping dan Parsing Teks Berita**

Menguraikan judul, URL link, konten, tanggal, tags dari laman artikel berita.

In [11]:
from bs4 import BeautifulSoup
from newspaper import Article
from selenium import webdriver
import time
from datetime import datetime

# Setup for Selenium (use your own options here)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")

def scrape_news_text(df_news_link):
    dict_news = dict()
    status = list()

    for i in df_news_link.index:
        # Akses dan download artikel
        article = Article(str(df_news_link['Link'][i]), language='id')
        article.download()

        # Jika download berhasil, uraikan HTML
        if article.download_state == 2:
            article.parse()

            # Uraikan HTML dengan BeautifulSoup
            soup = BeautifulSoup(article.html, 'html.parser')

            # Hapus elemen-elemen tertentu berdasarkan class atau tag
            for selector in ['div.photo__caption', 'strong.read__others', 'strong em', 'video_detail__title']:
                for tag in soup.select(selector):
                    tag.decompose()  # Menghapus elemen dari DOM

            # Hapus elemen video_detail__title beserta isinya
            video_detail = soup.find('div', class_='video_detail__title')
            if video_detail:
                video_detail.decompose()  # Hapus div beserta isinya

            # Set HTML yang sudah bersih
            article.set_html(str(soup))
            article.parse()

            # Ambil teks jika tersedia
            if article.text:
                print(f"{i+1} [success] {article.url}")
                status.append('success')

                # Mencari elemen tanggal dari HTML
                date_element = soup.find('meta', {'name': 'dtk:publishdate'})
                if date_element:
                    date_text = date_element['content'].strip()  # Format asli "YYYY/MM/DD HH:MM:SS"
                    # Ubah format tanggal menjadi DD/MM/YYYY dan hilangkan waktu
                    date_text = datetime.strptime(date_text, '%Y/%m/%d %H:%M:%S').strftime('%d/%m/%Y')
                else:
                    date_text = 'Unknown Date'

                dict_news[i] = {
                    'title': str(article.title),
                    'url': str(article.url),
                    'text': ' '.join(article.text.splitlines()),  # Gabungkan teks agar tidak terpisah
                    'date': date_text,  # Tanggal dalam format DD/MM/YYYY
                    'tags': ', '.join(article.tags)
                }
                continue

        # Jika gagal dengan Newspaper, gunakan Selenium
        driver = webdriver.Chrome(options=chrome_options)
        input_html = ''

        try:
            driver.get(str(df_news_link['Link'][i]))
            input_html = driver.page_source
            time.sleep(2)

            # Gunakan BeautifulSoup untuk mengekstrak tanggal dan teks
            soup = BeautifulSoup(input_html, 'html.parser')

            # Hapus elemen-elemen yang tidak diinginkan
            for selector in ['div.photo__caption', 'strong.read__others', 'strong em']:
                for tag in soup.select(selector):
                    tag.decompose()  # Menghapus elemen

            # Hapus elemen video_detail__title beserta isinya
            video_detail = soup.find('div', class_='video_detail__title')
            if video_detail:
                video_detail.decompose()  # Hapus div beserta isinya

            # Ambil tanggal dari meta tag atau div yang sesuai (disesuaikan dengan struktur baru)
            date_element = soup.find('meta', {'name': 'dtk:publishdate'})
            if date_element:
                date_text = date_element['content'].strip()  # Format asli "YYYY/MM/DD HH:MM:SS"
                # Ubah format tanggal menjadi DD/MM/YYYY dan hilangkan waktu
                date_text = datetime.strptime(date_text, '%Y/%m/%d %H:%M:%S').strftime('%d/%m/%Y')
            else:
                date_text = 'Unknown Date'

            # Gunakan Newspaper untuk menguraikan teks setelah HTML diperoleh melalui Selenium
            article.set_html(input_html)
            article.parse()

            if article.text:
                print(f"{i+1} [success] {article.url}")
                status.append('success')
                dict_news[i] = {
                    'title': str(article.title),
                    'url': str(article.url),
                    'text': ' '.join(article.text.splitlines()),
                    'date': date_text,  # Tanggal dalam format DD/MM/YYYY
                    'tags': ', '.join(article.tags)
                }
                continue
        except Exception as e:
            print(f"Error processing {df_news_link['Link'][i]}: {e}")
        finally:
            driver.close()

        # Jika gagal, beri tanda 'failed'
        print(f"{i+1} [failed] {df_news_link['Link'][i]}")
        status.append('failed')

    return dict_news, status


In [12]:
# Scraping dan parsing text berita
dict_news, status = scrape_news_text(df_news_link)

1 [success] https://www.cnbcindonesia.com/news/20240922222617-4-573678/kelas-123-bakal-dihapus-cek-iuran-bpjs-kesehatan-23-september-2024
2 [success] https://www.cnbcindonesia.com/news/20240922080603-4-573594/kelas-bpjs-1-2-3-dihapus-jokowi-cek-iuran-terkini
3 [success] https://www.cnbcindonesia.com/news/20240920071902-4-573199/bpjs-kesehatan-ungkap-3-rs-manipulasi-klaim-bikin-negara-boncos
4 [success] https://www.cnbcindonesia.com/lifestyle/20240919071855-33-572872/wajib-tahu-5-jenis-operasi-ini-tak-ditanggung-bpjs-kesehatan
5 [success] https://www.cnbcindonesia.com/news/20240917140717-4-572393/ri-siap-punya-bbm-baru-low-sulfur-luhut-sebut-bisa-tekan-subsidi-bpjs
6 [success] https://www.cnbcindonesia.com/news/20240917091811-4-572270/jokowi-hapus-kelas-bpjs-123-cek-iuran-per-17-september-2024
7 [success] https://www.cnbcindonesia.com/lifestyle/20240915181126-33-572029/duh-bos-bpjs-tk-ungkap-28-juta-pekerja-ri-rentan-jatuh-ekstrem
8 [success] https://www.cnbcindonesia.com/lifestyle/2024

# **Simpan Teks Berita ke CSV**

In [13]:
# Mengonversi dictionary ke dalam DataFrame
df_news = pd.DataFrame.from_dict(dict_news, orient='index')
df_status = pd.DataFrame(status, columns=['status'])

In [14]:
df_news.head(10)

,title,url,text,date,tags
0,"Kelas 1,2,3 Bakal Dihapus, Cek Iuran BPJS Kese...",https://www.cnbcindonesia.com/news/20240922222...,"Jakarta, CNBC Indonesia - Pemerintah akan meng...",23/09/2024,"#bpjs kesehatan, #iuran bpjs kesehatan"
1,"Kelas BPJS 1, 2, 3 Dihapus Jokowi, Cek Iuran T...",https://www.cnbcindonesia.com/news/20240922080...,"Jakarta, CNBC Indonesia - Skema iuran BPJS Kes...",22/09/2024,"#jokowi, #bpjs kesehatan, #bpjs, #iuran bpjs, ..."
2,"BPJS Kesehatan Ungkap 3 RS Manipulasi Klaim, B...",https://www.cnbcindonesia.com/news/20240920071...,"Jakarta, CNBC Indonesia - BPJS Kesehatan dan K...",20/09/2024,"#bpjs kesehatan, #rumah sakit, #kpk"
3,"Wajib Tahu, 5 Jenis Operasi Ini Tak Ditanggung...",https://www.cnbcindonesia.com/lifestyle/202409...,"Jakarta, CNBC Indonesia - Ada banyak layanan k...",19/09/2024,"#bpjs kesehatan, #operasi"
4,"RI Siap Punya BBM Baru Low Sulfur, Luhut Sebut...",https://www.cnbcindonesia.com/news/20240917140...,"Jakarta, CNBC Indonesia - Menteri Koordinator ...",17/09/2024,"#bbm baru low sulfur, #bbm baru, #menko marves..."
5,"Jokowi Hapus Kelas BPJS 1,2,3, Cek Iuran per 1...",https://www.cnbcindonesia.com/news/20240917091...,"Jakarta, CNBC Indonesia - Skema iuran BPJS Kes...",17/09/2024,"#bpjs kesehatan, #jokowi, #kelas bpjs kesehata..."
6,"Duh! Bos BPJS TK Ungkap 2,8 Juta Pekerja RI Re...",https://www.cnbcindonesia.com/lifestyle/202409...,"Jakarta, CNBC Indonesia - Badan Penyelenggara ...",15/09/2024,"#perlindungan pekerja, #bpjs ketenagakerjaan, ..."
7,21 Penyakit yang Pengobatannya Tak Gratis Mesk...,https://www.cnbcindonesia.com/lifestyle/202405...,"Jakarta, CNBC Indonesia - Badan Penyelenggara ...",12/09/2024,"#bpjs kesehatan, #penyakit, #penyakit tidak di..."
8,"RI Sedang Musim PHK, Tingkat Klaim BPJS TK Men...",https://www.cnbcindonesia.com/research/2024091...,"Jakarta, CNBC Indonesia - Melemahnya industri ...",12/09/2024,"#phk melonjak, #klaim bpjs tk, #bpjs ketenagak..."
9,Jangan Salah! 5 Jenis Operasi Ini Tak Ditanggu...,https://www.cnbcindonesia.com/lifestyle/202409...,"Jakarta, CNBC Indonesia - Belum banyak yang ta...",07/09/2024,"#bpjs kesehatan, #operasi tak ditanggung bpjs,..."


In [15]:
# Menyimpan DataFrame ke dalam file CSV
output_file = 'newstext_cnbc.csv'
df_news.to_csv(output_file, index=False, sep=";")
print(df_status['status'].value_counts())

status
success    1500
Name: count, dtype: int64
